In [9]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import webbrowser
import pickle
import json

In [ ]:
# json.loads(json문자열) : JSON 문자열 -> Dictionary
# json.dumps(dictionary): Dictionary -> JSON 문자열 

In [38]:
import pprint
with open('dart_api_key', 'rb') as f:
    auth_key = pickle.load(f)
company_code = '005380'    #현대차
start_date = '20170101'
dsp_tp = 'A' #공시문서 유형 = 정기공시(A), 주요사항보고(B), 발행공시(C)...
url = 'http://dart.fss.or.kr/api/search.json?auth={}&crp_cd={}&start_dt={}'.format(auth_key, company_code,start_date)

res = requests.get(url)
if res.status_code == 200:
    res_data = json.loads(res.text)    

In [39]:
crp_cls=[]
crp_cd=[]
rpt_nm=[]
rcp_no=[]
flr_nm=[]
rcp_dt=[]
rmk=[]

for data in res_data['list']:
    crp_cls.append(data['crp_cls'])
    crp_cd.append(data['crp_cd'])
    rpt_nm.append(data['rpt_nm'])
    rcp_no.append(data['rcp_no'])
    flr_nm.append(data['flr_nm'])
    rcp_dt.append(data['rcp_dt'])
    rmk.append(data['rmk'])

In [41]:
d = dict(
crp_cls=crp_cls,
crp_cd=crp_cd,
rpt_nm=rpt_nm,

    rcp_no=rcp_no,
flr_nm=flr_nm,
rcp_dt=rcp_dt,
rmk=rmk)

df = pd.DataFrame(d)
print(df.shape)
df.head()

(10, 7)


,crp_cls,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt,rmk
0,Y,005380,특수관계인과의내부거래,20191024000263,현대자동차,20191024,공
1,Y,005380,특수관계인에대한부동산임대,20191024000260,현대자동차,20191024,공
2,Y,005380,약관에의한금융거래시계열금융회사의거래상대방의공시,20191024000256,현대자동차,20191024,공
3,Y,005380,동일인등출자계열회사와의상품ㆍ용역거래,20191024000255,현대자동차,20191024,공
4,Y,005380,주요사항보고서(자기주식처분결정),20191024000251,현대자동차,20191024,


In [37]:
# rcp_no가 보고서 번호
# 보고서 조회 url : http://dart.fss.or.kr/dsaf001/main.do?rcpNo=보고서번호
report_url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}'
for rcp_no in df['rcp_no']:
    c_url = report_url.format(rcp_no)
    res = requests.get(c_url)
    print(res.text)
    break;




<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
<head>
<title>
현대자동차/특수관계인과의내부거래/2019.10.24
</title>
<meta http-equiv="X-UA-Compatible" content="IE=EmulateIE8" >
<meta http-equiv="Content-Type" content="text/html; charset=utf-8" >
<style type="text/css">
    .x-window-dlg .ext-mb-download {
        background:transparent url(images/download.gif) no-repeat top left;
        height:46px;
    }
</style>
<link rel="stylesheet" type="text/css" href="/css/viewer.css" charset="utf-8">
<script type="text/javascript" src="/js/prototype.js"></script>
<script type="text/javascript" src="/js/jquery/jquery-all.js"></script>

<!-- 2014.03.11 ext 3.4 -->
<!--[if lte IE 8]><link rel="stylesheet" type="text/css" href="/js/ext-viewer/resources/css/ext-all-ie8.css" ><![endif]-->
<!--[if (gte IE 9)|!(IE)]><!--><link rel="stylesheet" type="text/css" href="/js/ext-viewer/resources/css/ext-all.css" ><!--<![endif]-->
<script type="text/javascrip

공시문서 종류마다 페이지 형태가 다르므로 공시 문서별로 크롤링 해야 한다.
- TODO: 같은 종류 문서는 같은 형식인지 확인
- TODO: 문서종류별 조회는 어떻게 할 지 확인

In [48]:
#STEP 1
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import webbrowser


#STEP 2 
with open('dart_api_key', 'rb') as f:
    auth_key = pickle.load(f)
company_code = '005380'    
start_date = '20180101'

#STEP 3
url = "http://dart.fss.or.kr/api/search.xml?auth="+auth_key+"&crp_cd="+company_code+"&start_dt="+start_date+"&bsn_tp=A001&bsn_tp=A002&bsn_tp=A003"
# 'http://dart.fss.or.kr/api/search.json?auth={}&crp_cd={}&start_dt={}'.format(auth_key, company_code,start_date)
#STEP 4
resultXML=urlopen(url)  #this is for response of XML
result=resultXML.read() #Using read method

#STEP 5
xmlsoup=BeautifulSoup(result,'html.parser')

#STEP 6
data = pd.DataFrame()

te=xmlsoup.findAll("list")

for t in te:
    temp=pd.DataFrame(([[t.crp_cls.string,t.crp_nm.string,t.crp_cd.string,t.rpt_nm.string,
        t.rcp_no.string,t.flr_nm.string,t.rcp_dt.string, t.rmk.string]]),
        columns=["crp_cls","crp_nm","crp_cd","rpt_nm","rcp_no","flr_nm","rcp_dt","rmk"])
    data=pd.concat([data,temp])

#STEP 7 
data=data.reset_index(drop=True)

#OPTIONAL
print(data)
user_num=int(input("몇 번째 보고서를 확인하시겠습니까?"))
url_user="http://dart.fss.or.kr/dsaf001/main.do?rcpNo="+data['rcp_no'][user_num]
webbrowser.open(url_user)

  crp_cls crp_nm  crp_cd           rpt_nm          rcp_no flr_nm    rcp_dt  \
0       Y  현대자동차  005380  반기보고서 (2019.06)  20190814002230  현대자동차  20190814   
1       Y  현대자동차  005380  분기보고서 (2019.03)  20190515002011  현대자동차  20190515   
2       Y  현대자동차  005380  사업보고서 (2018.12)  20190401004132  현대자동차  20190401   
3       Y  현대자동차  005380  분기보고서 (2018.09)  20181114002234  현대자동차  20181114   
4       Y  현대자동차  005380  반기보고서 (2018.06)  20180814002199  현대자동차  20180814   
5       Y  현대자동차  005380  분기보고서 (2018.03)  20180515002108  현대자동차  20180515   
6       Y  현대자동차  005380  사업보고서 (2017.12)  20180402004162  현대자동차  20180402   

    rmk  
0  None  
1  None  
2     연  
3  None  
4  None  
5  None  
6     연  
몇 번째 보고서를 확인하시겠습니까?3


True

In [47]:
content = '''
<table>
<tbody>
<tr>
<td>265</td>
<td> <a href="/j/jones03.shtml">Jones</a>Blue</td>
<td >29</td>
</tr>
<tr >
<td>266</td>
<td> <a href="/s/smith01.shtml">Smith</a></td>
<td>34</td>
</tr>
</tbody>
</table>'''


lst = pd.read_html(content)
lst[0]

,0,1,2
0,265,JonesBlue,29
1,266,Smith,34
